In [8]:
import re, subprocess, pyodbc , json
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [32]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, GroupName, QuestionTypeCode, QuestionTypeID, QTEXT    
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df = df.loc[df.IdText != '']
df

,QPK,QGROUP,IdText,GroupName,QuestionTypeCode,QuestionTypeID,QTEXT
1,27001,3201,1.1,S1A,CIO2,39,"For each FIPS 199 impact level, what is the nu..."
2,27002,3201,1.1.1,S1A,LABEL,18,Organization-Operated Systems
3,27003,3201,1.1.2,S1A,LABEL,18,Contractor-Operated Systems
4,27004,3201,1.1.3,S1A,LABEL,18,Systems (from 1.1.1 and 1.1.2) with Security ATO
5,27005,3201,1.1.4,S1A,LABEL,18,Systems (from 1.1.3) that are in ongoing autho...
...,...,...,...,...,...,...,...
186,27434,3216,10.6.2,10,PICK,17,"If yes, did the agency identify changes necess..."
187,27435,3216,10.6.3,10,TXT,9,"If yes, what is the date when incident respons..."
188,27436,3216,10.6.4,10,YN,1,Does the agencys communications strategy inclu...
189,27437,3216,10.6.4.1,10,YN,1,Does the agencys communications strategy inclu...


In [27]:
def idtxt(r):
    tmp = """ DECLARE @ANS_$1 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='$2')"""
    tmp=tmp.replace('$1',r['IdText'].replace('.','_'))
    tmp=tmp.replace('$2',r['IdText'])
    return tmp

In [37]:
for i,r in df.loc[df.GroupName == '10'].iterrows(): 
    p = idtxt(r)
    print( p ) 

 DECLARE @ANS_10_1 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='10.1')
 DECLARE @ANS_10_1_1 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='10.1.1')
 DECLARE @ANS_10_1_2 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='10.1.2')
 DECLARE @ANS_10_1_3 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='10.1.3')
 DECLARE @ANS_10_1_4 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),0) END FROM @ListofQuestions2 WHERE identifier_text='10.1.4')
 DECLARE @ANS_10_2 INT = (SELECT CASE @PK_QuestionGroup WHEN 0 THEN TableAns ELSE COALESCE(CONVERT(INT,MetricAns),